In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

In [2]:
essays = requests.get('http://paulgraham.com/articles.html')

In [3]:
essays_soup = BeautifulSoup(essays.content, 'html.parser')

In [4]:
# do things that don't scale :P
essay_links = essays_soup.find('table').find_all('table')[1].find_all('a')

In [5]:
essay_pages = [
    (x.attrs['href'], x.text) for x in essay_links
    if not x.attrs['href'].startswith('https://')
]

In [6]:
def extract_data(page):
    url, title = page

    soup = BeautifulSoup(requests.get(f'http://paulgraham.com/{url}').content, 'html.parser')
    text = soup.find('table').find('font')
    text.attrs = {}
    text.name = 'div'
    
    return title, text

In [ ]:
essay_data = [extract_data(x) for x in tqdm(essay_pages)]

In [8]:
book = BeautifulSoup()
for title, text in essay_data.reverse():
    new_tag = book.new_tag('h1')
    new_tag.string = title
    book.body.append(new_tag)
    book.body.append(text)

In [9]:
with open("book.html", "w", encoding='utf-8') as file:
    file.write(str(book))